In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/NIR/ods_broadcasts_201905301157.csv.zip" /content/ods_broadcasts_201905301157.csv.zip
!cp "/content/drive/MyDrive/NIR/ods_match_news.csv.zip" /content/ods_match_news.csv.zip
!unzip /content/ods_broadcasts_201905301157.csv.zip
!unzip /content/ods_match_news.csv.zip

!cp "/content/drive/MyDrive/NIR/repository/summarization/news_after_match/generated_predictions.txt" "/content/after_match_generated_predictions.txt"
!cp "/content/drive/MyDrive/NIR/repository/summarization/news_during_match/generated_predictions.txt" "/content/during_match_generated_predictions.txt"

Archive:  /content/ods_broadcasts_201905301157.csv.zip
  inflating: ods_broadcasts_201905301157.csv  
Archive:  /content/ods_match_news.csv.zip
  inflating: ods_match_news.csv      


In [3]:
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
def remove_tags(html):
    soup = BeautifulSoup(str(html), "html.parser")
    
    for data in soup(['style', 'script']):
        data.decompose()

    return ' '.join(soup.stripped_strings)


data_broadcasts = pd.read_csv("/content/ods_broadcasts_201905301157.csv")

def concatenate_content(messages):
  return ' '.join([str(message).strip() for message in messages])

grouping_columns = ['match_id', 'team1', 'team2', 'name', 'match_time']
data_broadcasts = data_broadcasts.sort_values(['match_id', 'minute', 'message_time'], ascending = True).groupby(grouping_columns)['content'].apply(concatenate_content).reset_index()

data_broadcasts['content'] = data_broadcasts['content'].apply(remove_tags)

data_news = pd.read_csv('/content/ods_match_news.csv')

data_news['name'] = data_news['name'].apply(remove_tags)
data_news['body'] = data_news['body'].apply(remove_tags)

data = pd.merge(data_broadcasts, data_news, on="match_id", how="inner")
data.rename(columns = {'name_x' : 'broadcast_name', 'name_y' : 'news_name'}, inplace = True)

<ipython-input-4-0b622f75180d>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")
<ipython-input-4-0b622f75180d>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")


In [5]:
after_match_new_news, during_match_new_news = [], []

with open("/content/after_match_generated_predictions.txt", 'r', encoding='utf-8') as f:
  for line in f.readlines():
    after_match_new_news.append(line.strip('\n'))

with open("/content/during_match_generated_predictions.txt", 'r', encoding='utf-8') as f:
  for line in f.readlines():
    during_match_new_news.append(line.strip('\n'))

In [6]:
test_am = pd.read_csv("/content/Test_am.csv", encoding='utf-8')
test_im = pd.read_csv("/content/Test_im.csv", encoding='utf-8')

In [7]:
after_match_data = data[(data['id'].isin(test_am.news_id.tolist())) & (data['match_id'].isin(test_am.match_id.tolist()))]
after_match_data['input_content'] = test_am.content.tolist()
after_match_data['output_name_with_body'] = after_match_new_news
after_match_data

<ipython-input-7-208122d66226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_match_data['input_content'] = test_am.content.tolist()
<ipython-input-7-208122d66226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_match_data['output_name_with_body'] = after_match_new_news


,match_id,team1,team2,broadcast_name,match_time,content,id,news_name,ctime,body,input_content,output_name_with_body
10793,218334,Питтсбург,Монреаль,Кубок Стэнли,1273705200,"Привет всем, кто жить не может без НХЛ! Пробил...",70632454,НХЛ. Плей-офф. 1/2 финала конференций. Шестой ...,2010-05-12 20:12:46,Плей-офф 1/2 финала конференций Восточная конф...,1 минута. ...и уже на 10-й секунде отправил Дж...,«Питтсбург» – «Монреаль» – 1:4. Куниц установи...
12572,221115,Россия,Македония,Квалификация Евро,1314979200,Приветствуем Всех любителей самой популярной и...,120222055,Пандев получил красную карточку по окончании м...,2011-09-02 22:18:30,Нападающий «Наполи» и сборной Македонии Горан ...,"41 минута. гол! 1:0! Семшов! Разыграли, Аршави...",Россия – Македония – 1:0. Семшов забил победны...
12774,221166,Черногория,Англия,Квалификация Евро,1318014000,"Всем привет! Сборной Англии для того, чтобы со...",127529201,Англия пробилась в финальную стадию Евро-2012,2011-10-08 01:37:39,"Сборная Англии, сыграв вничью с Черногорией (2...","10 минута. гол! 0:1! Руни, находясь в центре, ...",«Черногория» – Англия – 2:2. Делибашич спас хо...
15053,232581,Спартак,ЦСКА,КХЛ,1287762300,"Привет всем, кто не представляет своей жизни б...",73759962,В матче со «Спартаком» Гайдученко сделал первы...,2010-10-22 22:21:28,21-летний голкипер ЦСКА Сергей Гайдученко в се...,11 минута. А удален Штястны за подножку. 17 ми...,«Спартак» – ЦСКА – 1:0. Шайба Штястны принесла...
16732,244493,СКА,Каролина,Выставочные игры КХЛ - НХЛ,1286206500,"Привет всем, кто жить не может без хоккея! Дав...",73427146,СКА впервые обыграл команду НХЛ,2010-10-04 22:28:25,"СКА, обыграв в выставочном матче «Каролину» (5...","2 минута. 1:0! Панов! Делал пас из-за ворот, ш...",«Каролина» – СКА – 2:2. Дуайер забросил победн...
...,...,...,...,...,...,...,...,...,...,...,...,...
87197,1047495,Вильярреал,Барселона,Испания. Ла Лига,1458486000,Всем привет! Продолжающая собирать скальпы «Ба...,1038323398,«Вилльяреал» – «Барселона» – 2:2. Команды сыгр...,2016-03-20 17:19:52,Сегодня в матче чемпионата Испании «Вильярреал...,16 минута. 2:0 повели каталонцы по предупрежде...,«Вильярреал» – «Барселона» – 2:0. Суарес забил...
87740,1063522,ЦСКА,Спортинг,Лига чемпионов,1440614700,Физкульт-привет! Финалисты Кубка УЕФА - 2005 с...,1032480309,ЦСКА – «Спортинг» – 3:1. Муса поставил точку в...,2015-08-26 10:53:37,Сегодня в ответном матче заключительного раунд...,35 минута. Набабкин меняет Фернандеса. В атаке...,«Спортинг» вышел в четвертьфинал Лиги чемпионо...
89601,1077519,Уфа,ЦСКА,Россия. Олимп Кубок России,1456840800,"Всем привет! Пришла весна, и вернулся российск...",1037641846,ЦСКА не побеждал в семи последних матчах,2016-03-01 15:48:55,Сегодня в ¼ финала Кубка России ЦСКА в гостях ...,55 минута. Первое предупреждение в ЦСКА : Алек...,«Уфа» – ЦСКА – 0:2. Голы Широкова и Мусы прине...
90089,1080240,Челси,ПСЖ,Лига чемпионов,1457552700,Всем привет! Продолжаем лигочемпионский вечер!...,1037899816,Лига чемпионов. 1/8 финала. Ответный матч. Гол...,2016-03-09 10:12:08,В ответном матче 1/8 финала Лиги чемпионов «Че...,"16 минута. гол! РАБЬО! Златан замахнулся и, шу...",«Челси» – «ПСЖ» – 1:2. Носатый забил победный ...


In [8]:
during_match_data = pd.merge(data, test_im, on="match_id", how="inner")
during_match_data = during_match_data[during_match_data['id'] == during_match_data['news_id']]
during_match_data.drop(['news_id', 'name', 'team1_y', 'team2_y', 'content_y', 'part_body'], axis=1, inplace=True)
during_match_data.rename(columns = {'team1_x' : 'team1', 'team2_x' : 'team2', 'content': 'content'}, inplace = True)
during_match_data['input_content'] = test_im.content.tolist()
during_match_data['output_name_with_body'] = during_match_new_news
during_match_data

,match_id,team1,team2,broadcast_name,match_time,content_x,id,news_name,ctime,body,input_content,output_name_with_body
3,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,"Добрый день! Sports.ru поздравляет всех, кто п...",7274151,Дюрица получил травму в игре с «Химками»,2009-03-14 17:48:18,Защитник «Локомотива» Ян Дюрица получил травму...,44 минута. Травмированного Дюрицу меняет Финин...,«Локомотив» – «Химки». Дюрицу получил травму. ...
14,19026,Динамо,Спартак,Россия. Премьер-лига,1252836000,"Привет! ""Динамо"" - ""Спартак"" - это вам не абы ...",31751785,Веллитон нанес травму Габулову,2009-09-13 14:34:52,Голкипер «Динамо» Владимир Габулов получил тра...,"45 минута. Хохлов принимает мяч в штрафной, оп...",«Динамо» – «Спартак». Габулов получил травму. ...
43,32607,Зенит,Амкар,Россия. Премьер-лига,1287231300,"Всем привет. Пока ""Зенит"" с каждым матчем приб...",73635945,Белоруков и Федорив получили красные карточки ...,2010-10-16 17:07:17,Защитники «Амкара» Дмитрий Белоруков и Виталий...,55 минута. Турбин! Турбин жесткоковыйный! Удал...,«Зенит» – «Амкар». Турбин удален с поля. Полуз...
64,32652,Локомотив,Зенит,Россия. Премьер-лига,1283083200,Приветствуем Всех любителей самой популярной и...,72692078,Бухаров забил первый гол за «Зенит»,2010-08-29 17:40:51,Форвард Александр Бухаров забил первый гол в с...,67 минута. И еще момент. Удар справа по ворота...,«Локомотив» – «Зенит». Бухаров забил первый го...
83,222616,Ливерпуль,Арсенал,Англия. Премьер-лига,1281884400,"""Ливерпуль"" с новым тренером и ""Арсенал"" с нов...",72385673,Коул и Косьельни получили красные карточки в м...,2010-08-15 21:00:04,Полузащитник «Ливерпуля» Джо Коул и защитник «...,45 минута. Джонсоооон! С левой метров с 20 соп...,«Ливерпуль» – «Арсенал». Коул удален с поля. З...
...,...,...,...,...,...,...,...,...,...,...,...,...
1844,1099582,ЦСКА,Металлург Мг,Кубок Гагарина,1460737800,"Привет всем, кто не представляет своей жизни б...",1039136054,ЦСКА – «Магнитка». Любимов не забил буллит в к...,2016-04-15 22:22:59,Нападающий ЦСКА Роман Любимов не реализовал бу...,65 минута. Макаров пытался бросать с левого кр...,«Магнитогорск» – ЦСКА. Мозякин забил 11-й гол ...
1933,1100353,Реал Мадрид,Атлетико,Лига чемпионов,1464461100,8 километров по прямой или 20 минут на автобус...,1040503616,«Реал» – «Атлетико». Гризманн не забил пенальти,2016-05-28 23:01:34,Форвард «Атлетико» Антуан Гризманн не реализов...,123 минута. Бэйл... Точно в угол! 3-2! 123 мин...,«Реал» – «Атлетико». Хуанфран забил первый гол...
1947,1100353,Реал Мадрид,Атлетико,Лига чемпионов,1464461100,8 километров по прямой или 20 минут на автобус...,1040507029,«Реал» – «Атлетико». Хуанфран попал в штангу в...,2016-05-29 00:32:44,"«Реал» одержал победу над «Атлетико» (1:1, 5:3...",120 минута. Лукас Васкес принимает мяч на своб...,«Реал» – «Атлетико». Хуанфран получил травму. ...
1949,1100353,Реал Мадрид,Атлетико,Лига чемпионов,1464461100,8 километров по прямой или 20 минут на автобус...,1040507029,«Реал» – «Атлетико». Хуанфран попал в штангу в...,2016-05-29 00:32:44,"«Реал» одержал победу над «Атлетико» (1:1, 5:3...",121 минута. Первым бьет Лукас Васкес. Точно в ...,«Реал» – «Атлетико». Гризманнн пропустит ответ...


In [9]:
during_match_data.to_csv('/content/during_match_test_data.csv', index=False, encoding='utf-8')
after_match_data.to_csv('/content/after_match_test_data.csv', index=False, encoding='utf-8')